In [1]:
from dotenv import load_dotenv
load_dotenv()

from agno.agent import Agent
from agno.models.azure  import AzureOpenAI
from agno.tools.reasoning import ReasoningTools
from agno.tools.yfinance import YFinanceTools


agent = Agent(
    model=AzureOpenAI(id="gpt-4.1-mini"),
    tools=[
        ReasoningTools(add_instructions=True),
        YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True, company_news=True),
    ],
    instructions=[
        "Use tables to display data",
        "Only output the report, no other text",
    ],
    markdown=True,
    monitoring=True,
)
agent.print_response("Write a report on MSFT", stream=True, show_full_reasoning=True, stream_intermediate_steps=True)

Output()

In [ ]:
from agno.agent import Agent
from agno.models.azure  import AzureOpenAI

from agno.playground import Playground, serve_playground_app
from agno.storage.sqlite import SqliteStorage
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools

agent_storage: str = "tmp/agents.db"

web_agent = Agent(
    name="Web Agent",
    model=AzureOpenAI(id="gpt-4.1-mini"),
    tools=[DuckDuckGoTools()],
    instructions=["Always include sources"],
    # Store the agent sessions in a sqlite database
    storage=SqliteStorage(table_name="web_agent", db_file=agent_storage),
    # Adds the current date and time to the instructions
    add_datetime_to_instructions=True,
    # Adds the history of the conversation to the messages
    add_history_to_messages=True,
    # Number of history responses to add to the messages
    num_history_responses=5,
    # Adds markdown formatting to the messages
    markdown=True,
)

finance_agent = Agent(
    name="Finance Agent",
    model=AzureOpenAI(id="gpt-4.1-mini"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True, company_news=True)],
    instructions=["Always use tables to display data"],
    storage=SqliteStorage(table_name="finance_agent", db_file=agent_storage),
    add_datetime_to_instructions=True,
    add_history_to_messages=True,
    num_history_responses=5,
    markdown=True,
)

app = Playground(agents=[web_agent, finance_agent]).get_app()

if __name__ == "__main__":
    serve_playground_app("stock_analyzer:app", reload=True)


INFO Starting playground on http://localhost:7777

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Agent Playground ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                             ┃
┃                                                                             ┃
┃  Playground URL: ]8;id=691347;https://app.agno.com/playground?endpoint=localhost%3A7777\https://app.agno.com/playground?endpoint=localhost%3A7777]8;;\  ┃
┃                                                                             ┃
┃                                                                             ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

INFO:     Will watch for changes in these directories: ['/home/ythuang/workspace/agent-learn/agno-learn']
INFO:     Uvicorn running on http://localhost:7777 (Press CTRL+C to quit)
INFO:     Started reloader process [73909] using StatReload
ERROR:    Error loading ASGI app. Could not import module "stock_analyzer".
INFO:     Stopping reloader process [73909]
